# Ex sheet 1: simulation of random variables

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as rd

## The inversion method

### Generate exponential and geometric r.v.'s

In [ ]:
### Generate exponential distributed random variables given the mean and number of random variables
def exponential_inverse_trans(n=1,mean=1):
    U=rd.uniform(size=n)
    X=-mean*np.log(1-U)
    actual=rd.exponential(size=n,scale=mean)
    
    plt.figure(figsize=(14,6))
    plt.hist(X, bins=50, alpha=0.5, label="Generated r.v.")
    plt.hist(actual, bins=50, alpha=0.5, label="Actual r.v.")
    plt.title("Generated vs Actual %i Exponential Random Variables" %n)
    plt.legend()
    plt.show()
    return X

In [ ]:
expo_example1=exponential_inverse_trans(n=1000,mean=1)
expo_example2=exponential_inverse_trans(n=10000,mean=1)
expo_example3=exponential_inverse_trans(n=100000,mean=1)

In [ ]:
### Generate geometric distributed random variables given the parameter and number of random variables
def geometric_inverse_trans(n=1,param=0.5):
    U=rd.uniform(size=n)
    X=np.ceil(np.log(1-U)/np.log(1-param))
    actual=rd.geometric(p=param, size=n)
    
    plt.figure(figsize=(14,6))
    plt.hist(X, alpha=0.5, label="Generated r.v.")
    plt.hist(actual, alpha=0.5, label="Actual r.v.")
    plt.title("Generated vs Actual %i Geometric Random Variables" %n)
    plt.legend()
    plt.show()
    return X

In [ ]:
geom_example1=geometric_inverse_trans(n=1000,param=0.25)
geom_example2=geometric_inverse_trans(n=10000,param=0.5)
geom_example3=geometric_inverse_trans(n=100000,param=0.75)

## Generate r.v.'s with a density using the rejection method

In [ ]:
# Define the rejection method that returns a r.v. with the density wanted ≤ Cst*reference and the number of trials
def rejection_method(wanted,reference,Cst):
    res=np.zeros(2)
    while True:
        res[1]+=1
        X=rd.exponential(size=1,scale=1) # Rem: here we use that we know how to sample from our density reference
        U=rd.uniform(low=0, high=Cst*reference(X))
        if U < wanted(X) :
            res[0]=X
            return res

# Sample n i.i.d. r.'v.'s with this method
def sample_reject(wanted,reference,Cst,n=1):
    sample=[]
    trials=[]
    res=np.zeros((2,n))
    for i in range(n):
        temp=rejection_method(wanted,reference,Cst)
        sample.append(temp[0])
        trials.append(temp[1])
    res[0]=sample
    res[1]=trials
    return res

In [ ]:
# Define the reference density function, here a standard exponential
def density_reference(x):
    return np.exp(-x)

# Define the density function we want to simulate, here the absolute value of a standard Gaussian
def density_wanted(x):
    return math.sqrt(2/math.pi)*np.exp(-x**2/2)

# Define the constant that will give the average number of trials
cst=np.sqrt(2*math.e/math.pi)

# Plot the wanted density
xs = np.linspace(0, 5, 1000)
ys = density_wanted(xs)

plt.figure(figsize=(14,6))
plt.plot(xs, ys, label="Wanted density") 
plt.fill_between(xs, ys, 0, alpha=0.2)
plt.xlabel("x"), plt.ylabel("y"), plt.legend();

In [ ]:
n=1000
temp = sample_reject(density_wanted,density_reference,cst,n)
samps = temp[0]
trials = temp[1]
actual = np.abs(rd.normal(0,1,size=n))

np.mean(trials)
# Theory says sqrt(2e/pi), approx. 1.32

In [ ]:
n=100000
temp = sample_reject(density_wanted,density_reference,cst,n)
samps = temp[0]
trials = temp[1]
actual=np.abs(rd.normal(0,1,size=n))

plt.figure(figsize=(14,6))
plt.plot(xs, ys, label="Density function")
plt.hist(samps, bins=100, density=True, alpha=0.5, label="Sample distribution")
# plt.hist(actual, bins=100, density=True, alpha=0.5, label="Actual distribution")
plt.legend();

In [ ]:
# Directly sample n i.i.d. r.'v.'s with the density wanted ≤ Cst*reference

def batch_rejection_method(wanted,reference,Cst, num_samples, n=1):
    samples = []
    while len(samples) < num_samples:
        X=rd.exponential(scale=1, size=n) 
        # Rem: here we use that we know how to sample from our density_reference
        U=rd.uniform(low=0, high=Cst*reference(X), size=n)
        samples += X[U < wanted(X)].tolist()
    return samples[:num_samples]

In [ ]:
n=100000
samps = batch_rejection_method(density_wanted,density_reference,cst,n)
actual=np.abs(rd.normal(0,1,size=n))

plt.figure(figsize=(14,6))
plt.plot(xs, ys, label="Density function")
plt.hist(samps, bins=100, density=True, alpha=0.5, label="Sample distribution")
plt.hist(actual, bins=100, density=True, alpha=0.5, label="Actual distribution")
plt.legend();

# Box-Müller for sampling Gaussian r.v.'s

In [ ]:
# Define the radius, with the exponential law(1/2) and anlge, uniform(0,2*pi)
def rad(n=1):
    return np.sqrt(rd.exponential(scale=2, size=n))

def ang(n=1):
    return rd.uniform(low=0, high=2*math.pi, size=n)

In [ ]:
# Define the Gaussian density function we want to simulate
def gaussian(x):
    return 1/math.sqrt(2*math.pi)*np.exp(-x**2/2)

xs = np.linspace(-5, 5, 1000)
ys = gaussian(xs)

plt.figure(figsize=(6,6))
plt.plot(xs, ys, label="Gaussian density") 
plt.fill_between(xs, ys, 0, alpha=0.2)
plt.legend();

In [ ]:
n=100000
X=np.multiply(rad(n),np.cos(ang(n)))
Y=np.multiply(rad(n),np.sin(ang(n)))

plt.figure(figsize=(16,6))
plt.subplot(1, 2, 1)
plt.plot(xs, ys, label="Gaussian density")
plt.hist(X, bins=100, density=True, alpha=0.5, label="Sample of X's")
plt.legend();

plt.subplot(1, 2, 2)
plt.plot(xs, ys, label="Gaussian density")
plt.hist(Y, bins=100, density=True, alpha=0.5, label="Sample of Y's", color = 'g')
plt.legend();